In [246]:
import pandas as pd
import numpy as np

In [247]:
# 定义相对路径
file_data_dictionary = 'data/data_dictionary.csv'
file_summerOly_athletes = 'data/summerOly_athletes.csv'
file_summerOly_hosts = 'data/summerOly_hosts.csv'
file_summerOly_medal_counts = 'data/summerOly_medal_counts.csv'
file_summerOly_programs = 'data/summerOly_programs.csv'
file_countries = 'ProcessedData/Countries.csv'

# 读取表格
df_data_dictionary = pd.read_csv(
    file_data_dictionary,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)

df_summerOly_athletes = pd.read_csv(
    file_summerOly_athletes,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
# 筛选并修改 'NOC' 列中 LBA 为 LBI
df_summerOly_athletes.loc[df_summerOly_athletes['NOC'] == 'LBA', 'NOC'] = 'LBI'
# 筛选并修改 'NOC' 列中 ROC 为 RUS
df_summerOly_athletes.loc[df_summerOly_athletes['NOC'] == 'ROC', 'NOC'] = 'RUS'
# 创建映射字典
replacement_map = {
    'North Macedonia': 'Macedonia',
    'Taiwan': 'Chinese Taipei',
    'FR Yugoslavia': 'Yugoslavia',
    'Formosa': 'Chinese Taipei',
    'Ceylon': 'Sri Lanka',
    'British West Indies': 'Great Britain'
}
# 使用 replace 方法替换
df_summerOly_athletes['Team'] = df_summerOly_athletes['Team'].replace(replacement_map)

df_summerOly_hosts = pd.read_csv(
    file_summerOly_hosts,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)

df_summerOly_medal_counts = pd.read_csv(
    file_summerOly_medal_counts,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
df_summerOly_medal_counts['NOC'] = df_summerOly_medal_counts['NOC'].replace(replacement_map)

df_summerOly_programs = pd.read_csv(
    file_summerOly_programs,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
Hoster = pd.read_csv('ProcessedData/host_Country.csv')
Country = pd.read_csv('ProcessedData/Countries.csv')

In [248]:
event_count = df_summerOly_athletes[['Sport','Event','Year','NOC']].drop_duplicates()
event_count = event_count.assign(Event=event_count["Event"].str.split(", ")).explode("Event")
event_count = event_count.assign(Sport=event_count["Sport"].str.split(", ")).explode("Sport")
event_count = event_count.assign(Sport=event_count["Sport"].str.split("/")).explode("Sport").drop_duplicates()
event_count

,Sport,Event,Year,NOC
0,Basketball,Basketball Men's Basketball,1992,CHN
1,Judo,Judo Men's Extra-Lightweight,2012,CHN
2,Football,Football Men's Football,1920,DEN
3,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1900,DEN
4,Athletics,Athletics Women's 100 metres,1932,NED
...,...,...,...,...
252537,Athletics,Men's 100m,2024,CAY
252555,Swimming,Men's 100m Breaststroke,2024,LAO
252558,Judo,Women +78 kg,2024,UKR
252560,Athletics,Women's 100m,2024,GEQ


In [249]:
sport_event_summary = event_count.groupby("Sport").agg(
    Event_Count=("Event", "count"),  # 统计 Event 的总数量
    Event_Types=("Event", lambda x: ", ".join(sorted(set(x))))  # 列举所有唯一 Event，用逗号分隔
).reset_index()
sport_event_summary 

,Sport,Event_Count,Event_Types
0,3x3 Basketball,32,"Men, Men Team, Women, Women Team"
1,Aeronautics,1,Aeronautics Mixed Aeronautics
2,Alpinism,2,Alpinism Mixed Alpinism
3,Archery,1269,"28 metres, 33 metres, 50 metres, Archery Men's..."
4,Art Competitions,739,"Applied Arts, Architectural Designs, Art Compe..."
...,...,...,...
65,Tug-Of-War,22,Tug-Of-War Men's Tug-Of-War
66,Volleyball,340,"Men, Men Team, Volleyball Men's Volleyball, Vo..."
67,Water Polo,399,"Men, Men Team, Water Polo Men's Water Polo, Wa..."
68,Weightlifting,3618,"Men's +102kg, Men's +109kg, Men's 102kg, Men's..."


In [250]:
event_medal_count = df_summerOly_athletes[['Sport','Event','Year','NOC','Medal']].drop_duplicates()
event_medal_count = event_medal_count.assign(Event=event_medal_count["Event"].str.split(", ")).explode("Event")
event_medal_count = event_medal_count.assign(Sport=event_medal_count["Sport"].str.split(", ")).explode("Sport")
event_medal_count = event_medal_count.assign(Sport=event_medal_count["Sport"].str.split("/")).explode("Sport").drop_duplicates()
# event_medal_count = pd.merge(event_medal_count,isHoster,on= ['NOC','Year'],left_index=False,right_index=False)
event_medal_count

,Sport,Event,Year,NOC,Medal
0,Basketball,Basketball Men's Basketball,1992,CHN,No medal
1,Judo,Judo Men's Extra-Lightweight,2012,CHN,No medal
2,Football,Football Men's Football,1920,DEN,No medal
3,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1900,DEN,Gold
4,Athletics,Athletics Women's 100 metres,1932,NED,No medal
...,...,...,...,...,...
252545,Athletics,Women's 400m Hurdles,2024,USA,No medal
252555,Swimming,Men's 100m Breaststroke,2024,LAO,No medal
252558,Judo,Women +78 kg,2024,UKR,No medal
252560,Athletics,Women's 100m,2024,GEQ,No medal


In [251]:
event_medal_count_noNomedal = event_medal_count[event_medal_count['Medal'] != 'No medal']
event_medal_count_noNomedal.drop_duplicates()

,Sport,Event,Year,NOC,Medal
3,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1900,DEN,Gold
12,Swimming,Swimming Men's 200 metres Breaststroke,1920,FIN,Bronze
13,Swimming,Swimming Men's 400 metres Breaststroke,1920,FIN,Bronze
15,Gymnastics,Gymnastics Men's Individual All-Around,1948,FIN,Bronze
16,Gymnastics,Gymnastics Men's Team All-Around,1948,FIN,Gold
...,...,...,...,...,...
252331,Wrestling,Men's Greco-Roman 67kg,2024,AZE,Bronze
252336,Wrestling,Men's Freestyle 97kg,2024,AZE,Bronze
252367,Taekwondo,Women -57kg,2024,BUL,Bronze
252390,Wrestling,Men's Freestyle 74kg,2024,ALB,Bronze


In [252]:
event_medal_count = event_medal_count.merge(
    Hoster[['NOC', 'Year', 'isHoster']],  # 提取 NOC, Year, isHoster 列
    on=['NOC', 'Year'],  # 按 NOC 和 Year 匹配
    how='left'  # 左连接，保留 event_medal_count 中的所有行
).drop_duplicates()
event_medal_count[['isHoster']] = event_medal_count[['isHoster']].fillna(0).astype(int)

event_medal_count

,Sport,Event,Year,NOC,Medal,isHoster
0,Basketball,Basketball Men's Basketball,1992,CHN,No medal,0
1,Judo,Judo Men's Extra-Lightweight,2012,CHN,No medal,0
2,Football,Football Men's Football,1920,DEN,No medal,0
3,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1900,DEN,Gold,0
4,Athletics,Athletics Women's 100 metres,1932,NED,No medal,0
...,...,...,...,...,...,...
130761,Athletics,Women's 400m Hurdles,2024,USA,No medal,0
130762,Swimming,Men's 100m Breaststroke,2024,LAO,No medal,0
130763,Judo,Women +78 kg,2024,UKR,No medal,0
130764,Athletics,Women's 100m,2024,GEQ,No medal,0


In [253]:
medal_countsBySport = (
    event_medal_count
    .groupby(['NOC', 'Year', 'Sport','isHoster', 'Medal'])
    .size()
    .unstack(fill_value=0)  # 展开 Medal 列，并填充缺失值为 0
)

# 如果需要将列名变为 Gold、Silver、Bronze 和 No medal（可选）
medal_countsBySport = medal_countsBySport.rename_axis(columns=None).reset_index().drop_duplicates()
medal_countsBySport.to_csv('ProcessedData/medalCountsBySport.csv')
medal_countsBySport

,NOC,Year,Sport,isHoster,Bronze,Gold,No medal,Silver
0,AFG,1936,Athletics,0,0,0,3,0
1,AFG,1936,Hockey,0,0,0,1,0
2,AFG,1948,Football,0,0,0,1,0
3,AFG,1948,Hockey,0,0,0,1,0
4,AFG,1956,Hockey,0,0,0,1,0
...,...,...,...,...,...,...,...,...
26217,ZIM,2020,Rowing,0,0,0,1,0
26218,ZIM,2020,Swimming,0,0,0,2,0
26219,ZIM,2024,Athletics,0,0,0,3,0
26220,ZIM,2024,Rowing,0,0,0,1,0


In [254]:
medal_countsByEvent = (
    event_medal_count
    .groupby(['NOC', 'Year', 'Event','isHoster', 'Medal'])
    .size()
    .unstack(fill_value=0)  # 展开 Medal 列，并填充缺失值为 0
)

# 如果需要将列名变为 Gold、Silver、Bronze 和 No medal（可选）
medal_countsByEvent = medal_countsByEvent.rename_axis(columns=None).reset_index().drop_duplicates()
medal_countsByEvent.to_csv('ProcessedData/medalCountsByEvent.csv')
medal_countsByEvent

,NOC,Year,Event,isHoster,Bronze,Gold,No medal,Silver
0,AFG,1936,Athletics Men's 100 metres,0,0,0,1,0
1,AFG,1936,Athletics Men's Long Jump,0,0,0,1,0
2,AFG,1936,Athletics Men's Shot Put,0,0,0,1,0
3,AFG,1936,Hockey Men's Hockey,0,0,0,1,0
4,AFG,1948,Football Men's Football,0,0,0,1,0
...,...,...,...,...,...,...,...,...
115756,ZIM,2024,Men's 200m Backstroke,0,0,0,1,0
115757,ZIM,2024,Men's Marathon,0,0,0,1,0
115758,ZIM,2024,Men's Single Sculls,0,0,0,1,0
115759,ZIM,2024,Women's 100m Freestyle,0,0,0,1,0
